### Exercises

#### Question 1

Alongside this notebook is a data file named `daily_quotes.csv` which contains EOD OHLC/Volume data for a small number of equities over a 6 month period.

The first step is to load up this data into a dataframe, ensuring that all data types are correct (datetime objects for dates, floats for OHLC data, and integers for Volume).

Write a function that receives the file name as an argument and returns a dataframe that:
- has the correct data type for each column (`str`, `float`, `int`)
- has a row index based on the `symbol` column

In addition, we would like our dataframe to contain columns named and ordered in a specific way:
- symbol (`str`)
- date (`datetime`)
- open (`float`)
- high (`float`)
- low (`float`)
- close (`float`)
- volume (`int`)

(with `symbol` being used as the row index)

Hint: 

You will want to read up the Pandas docs for `read_csv` to see how you can handle datetime data directly while loading the data (in particular you should look at the `parse_dates` option):

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

Alternatively, you could convert these objects into proper datetime types after loading by using the Pandas function `to_datetime`, documented here:

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html

and then use conatenation to build up a dataframe that replaces the "old" `date` column with the "new" (properly typed) column.

In [1]:
import pandas as pd

In [2]:
def get_data_frame(path):
    df = pd.read_csv(
        path,
        header=0,
        names=['symbol', 'date', 'close', 'volume', 'open', 'high', 'low'],
        index_col = "symbol",
        parse_dates=["date"]
    )
    df = df.loc[:, ["date", "open", "high", "low", "close", "volume"]]
    return df

get_data_frame("daily_quotes.csv")

C:\Users\swapn\AppData\Local\Temp\ipykernel_16948\1787128040.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(


,date,open,high,low,close,volume
symbol,,,,,,
AAPL,2021-02-12,134.350,135.5300,133.6921,135.37,60145130
AMZN,2021-02-12,3250.000,3280.2500,3233.3100,3277.71,2335339
GOOG,2021-02-12,2090.250,2108.8200,2083.1300,2104.11,855865
MSFT,2021-02-12,243.933,245.3000,242.7300,244.99,16561080
AAPL,2021-02-11,135.900,136.3900,133.7700,135.13,64280030
...,...,...,...,...,...,...
MSFT,2020-08-14,208.760,209.5900,207.5100,208.90,17958940
AAPL,2020-08-13,114.430,116.0425,113.9275,115.01,210082080
AMZN,2020-08-13,3182.990,3217.5211,3155.0000,3161.02,3149043


#### Question 2

Write a function that, given a dataframe sructured as the one we created in Question 1 and a symbol name as a string (e.g. `AAPL`, `MSFT`, etc), will:
- return a similarly structured dataframe consisting of the row (or rows) containing the records with the highest volume for the given symbol
- raises a `ValueError` if the symbol is not in the dataframe

In [3]:
def max_volume(path, symbol):
    df = pd.read_csv(
        path,
        header=0,
        names=['symbol', 'date', 'close', 'volume', 'open', 'high', 'low'],
        index_col = "symbol",
        parse_dates=["date"]
    )
    df = df.loc[:, ["date", "open", "high", "low", "close", "volume"]]

    try:
        df = df.loc[symbol]
        max_vol = df.loc[:, "volume"].max()
        return df.loc[df.loc[:, "volume"] == max_vol]

    except KeyError as ex:
        print("KeyError:", ex)

max_volume("daily_quotes.csv", "AAPL")

C:\Users\swapn\AppData\Local\Temp\ipykernel_16948\871895027.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(


,date,open,high,low,close,volume
symbol,,,,,,
AAPL,2020-08-24,128.6975,128.785,123.9363,125.8575,345937760


P.#### Question 3

Using the same dataframe as in the preceding questions, our goal now is to write a function that will return, for a specific symbol, the row that had the largest high-low spread.

Write a function to do that - it should just return a dataframe with the row (or rows) with the largest high-low spread.

In [4]:
def max_high_low_spread(path, symbol):
    df = pd.read_csv(
        path,
        header=0,
        names=['symbol', 'date', 'close', 'volume', 'open', 'high', 'low'],
        index_col = "symbol",
        parse_dates=["date"]
    )
    df = df.loc[:, ["date", "open", "high", "low", "close", "volume"]]

    try:
        df = df.loc[symbol]
        spread = df.loc[:, "high"] - df.loc[:, "low"]
        max_spread = (df.loc[:, "high"] - df.loc[:, "low"]).max()
        df = df.loc[spread == max_spread]
        return df

    except KeyError as ex:
        print("KeyError:", ex)

max_high_low_spread("daily_quotes.csv", "AAPL")

C:\Users\swapn\AppData\Local\Temp\ipykernel_16948\3656309994.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(


,date,open,high,low,close,volume
symbol,,,,,,
AAPL,2020-09-04,120.07,123.7,110.89,120.96,332607200


#### Question 4

Using the same dataframe as the preceding questions, write a function that returns a single dataframe containing the record(s) with maximum high-low spread for each symbol in the dataframe. (Do not hardcode symbol names in this function - instead you should recover the possible symbol names from the data itself).

The returned dataframe should have the same structure as the original dataframe, but just contain the rows of maximum high-low spread for each symbol.

In [5]:
import pandas as pd
import numpy as np

In [6]:
def max_high_low_spread_all(path):
    df = pd.read_csv(
        path,
        header=0,
        names=['symbol', 'date', 'close', 'volume', 'open', 'high', 'low'],
        index_col="symbol",
        parse_dates=["date"]
    )[["date", "open", "high", "low", "close", "volume"]]

    res = pd.DataFrame()

    for symbol in df.index.unique():
        current = df.loc[symbol]
        ans = current.loc[current.loc[:, "high"] - current.loc[:, "low"] == (current.loc[:, "high"] - current.loc[:, "low"]).max()]

        if not res.empty:
            res = pd.concat([res, ans])

        else:
            res = ans

    return res

max_high_low_spread_all("daily_quotes.csv")

C:\Users\swapn\AppData\Local\Temp\ipykernel_16948\2908189215.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(


,date,open,high,low,close,volume
symbol,,,,,,
AAPL,2020-09-04,120.07,123.70,110.8900,120.96,332607200
AMZN,2020-09-04,3318.00,3381.50,3111.1300,3294.62,8781754
GOOG,2021-02-03,2073.00,2116.50,2018.3800,2070.07,4118170
MSFT,2020-09-03,229.27,229.31,214.9602,217.30,58400290
